In [2]:
from CodonOpt import *

In [3]:
amylase1 = 'ATGAAATTCTTCCTGCTGCTTTCCCTCATTGGATTCTGCTGGGCCCAATATGACCCACATACTCAATATGGACGAACTGCTATTGTCCACCTGTTTGAGTGGCGCTGGGTTGATATTGCTAAGGAATGTGAGAGATACTTAGCTCCTAATGGATTTGCAGGTGTGCAGGTCTCTCCACCCAATGAAAACATCGTAGTCCACAGCCCTTCAAGACCATGGTGGGAAAGATATCAACCAATTAGCTACAAAATATGTTCCAGGTCTGGAAATGAAGATGAATTCAGGGACATGGTGAACAGGTGCAACAATGTTGGTGTCCGTATTTATGTGGATGCTGTCATTAACCACATGTGTGGAGTGGGGGCTCAAGCTGGACAAAGCAGTACATGTGGAAGTTATTTCAACCCAAATAACAGGGACTTTCCTGGAGTTCCCTATTCTGGTTTTGACTTTAATGATGGAAAATGTAGAACTGCAAGTGGAGGTATCGAGAACTACCAAGATGCTGCTCAGGTCAGAGATTGTCGTCTGTCTGGCCTTCTGGATCTTGCACTTGAGAAAGATTATGTTCGAACCAAGGTGGCTGACTATATGAACCATCTCATTGACATTGGCGTAGCAGGGTTCAGACTTGATGCTTCTAAGCACATGTGGCCTGGAGACATAAAGGCAATTTTGGACAAACTGCATAATCTCAATACAAAATGGTTCTCCCAAGGAAGCAGACCTTTCATTTTCCAAGAGGTGATTGATCTGGGTGGTGAGGCAGTGTCAAGTAATGAGTATTTTGGAAATGGCCGTGTGACAGAATTCAAATATGGAGCAAAATTGGGCAAAGTTATGCGCAAGTGGGATGGAGAAAAGATGTCCTACTTAAAGAACTGGGGAGAAGGTTGGGGTTTGATGCCTTCTGACAGAGCCCTTGTGTTTGTGGACAACCATGACAATCAGCGAGGACATGGTGCTGGGGGAGCATCCATCTTGACATTCTGGGATGCTAGACTCTATAAAATGGCTGTTGGCTTTATGTTGGCTCATCCTTATGGTTTCACACGGGTGATGTCAAGTTACTATTGGCCAAGAAATTTCCAGAATGGAAAAGATGTCAATGACTGGGTTGGACCACCAAATAACAATGGAAAAACCAAAGAAGTGAGCATTAACCCAGACAGCACTTGTGGCAATGACTGGATCTGTGAACATCGATGGCGTCAAATAAGGAACATGGTTGCCTTCAGAAATGTCGTCAATGGTCAGCCTTTTGCAAACTGGTGGGATAATGACAGCAACCAGGTAGCTTTTGGCAGAGGAAACAAAGGATTCATTGTCTTTAACAATGATGACTGGGCTTTGTCAGAAACTTTACAGACTGGTCTTCCTGCTGGCACATACTGTGATGTCATTTCTGGAGATAAAGTCGATGGCAATTGCACTGGAATAAAAGTCTATGTTGGCAATGATGGCAAAGCTCACTTTTCTATTAGTAACTCTGCCGAAGACCCATTTATTGCAATCCATGCAGAGTCAAAAATATAA'
compute_CAI(amylase1,codon_freq)

0.7578865327134019

In [ ]:
#Compute the CAI for each gene
CAI = []
for cds in genes['cds']:
    CAI.append(compute_CAI(cds,codon_freq))
    
genes['CAI'] = CAI 

In [ ]:
genes.to_csv('../data/CAI.csv', index=True)